# Libraries

본 실습에서는 다음 두개의 Python 라이브러리들을 사용합니다.

1. tqdm: 프로그레스바를 표시해 주는 라이브러리 ([참조링크](https://github.com/tqdm/tqdm))

1. pytube: YouTube에서 video 컨텐츠를 다운로드 할 수 있도록 해주는 라이브러리 ([참조링크](https://pytube.io/en/latest/index.html))

## tqdm Overview

In [ ]:
from tqdm import tqdm
from tqdm import trange
from time import sleep

In [ ]:
# 다음 코드는 0.05초 마다 한번 씩 프로그레스 바를 업데이트 합니다.
for _ in tqdm(range(100)):
    sleep(0.05)

In [ ]:
# tqdm을 객체로 선언하고 update()를 호출해서 프로그레스를 업데이트 할 수도 있습니다.
pbar = tqdm(total=100)

for _ in range(10):
    sleep(0.1)
    pbar.update(10)
pbar.close()


In [ ]:
# 혹은, with keyword를 사용해서 close() 호출을 없앨 수도 있습니다.
with tqdm(total=100) as qbar:
    for _ in range(10):
        sleep(0.1)
        qbar.update(10)

## pytube overview
시간이 오래 걸리는 일을 모사하기 위해 pytube로 YouTube video를 다운로드 하는 경우를 가정했습니다.


In [ ]:
from pytube import YouTube
from IPython.display import Image

In [ ]:
# YouTube clip links

# 과학쿠키 - 초전도체
super_conductor = "https://www.youtube.com/watch?v=d_F7MVuJJu8"

# 8마일 랩배틀 에미넴모음, 연령제한
eight_miles = "https://www.youtube.com/watch?v=tWFejQSKIYg"

# IVE 아이브 'I AM' MV 
ive_mv = "https://www.youtube.com/watch?v=6ZUIwj3FgUY&list=PLObJu56es5eN3sNytFyk7CA3HKgXoRvf6"

In [ ]:

url = super_conductor
youtube_clip = YouTube(url)

In [ ]:
# Video clip에 대한 더 많은 정보는 다음의 링크를 참조 하세요.
# https://pytube.io/en/latest/api.html?highlight=author#youtube-object
print(f"제목:\t{youtube_clip.title}")
print(f"설명:\t" "<없음>" if youtube_clip.description is None else "f{youtube_clip.description}")
print(f"길이:\t약 {youtube_clip.length//60}분")
try:
    # Will reaises AgeRestrictedError if it's age restricted.
    # which is more accurate than youble_clip._age_restricted
    #youtube_clip.bypass_age_gate()
    print(f"연령제한 없는 컨텐츠")
except Exception:
    print(f"연령제한 있음")
Image(url=youtube_clip.thumbnail_url, width=500)

In [ ]:
# Streams
for i, item in enumerate(youtube_clip.streams):
    print(f"[1]: {item}")


In [ ]:
# 특정한 비디오 스트림을 지정해서 다운로드를 수행합니다. 필터링하는 다양한 방법들은 아래의 링크를 참조하세요.
# https://pytube.io/en/latest/api.html?highlight=highest%20resolution#streamquery-object

download_dir = "videos"

# E.g. download the first in progressive mp4 file.
#youtube_clip.streams.filter(progressive=True, file_extension='mp4').first().download(download_dir)

# E.g. download a highest resolution video.
youtube_clip.streams.get_highest_resolution().download(download_dir)

### Callbacks

다운로드의 진행과 완료때 호출 될 callback을 지정할 수 있습니다. 다음과 같이 생성자에서 on_progress_callback 혹은 on_complete_callback에 callback 함수를 연결해 주면 됩니다.

```
class YouTube:
    """Core developer interface for pytube."""

    def __init__(
        self,
        url: str,
        on_progress_callback: Optional[Callable[[Any, bytes, int], None]] = None,
        on_complete_callback: Optional[Callable[[Any, Optional[str]], None]] = None,
        proxies: Dict[str, str] = None,
        use_oauth: bool = False,
        allow_oauth_cache: bool = True
    ):
```

In [ ]:
# 출시 1년도 되지 않은 카트라이더 드리프트는 왜 망해갈까
video_url = "https://www.youtube.com/watch?v=r8y7RjlhkNE"

# Progress callback
def on_progress(stream, chunk, bytes_remaining):
    print(f"Progress: about {bytes_remaining // (1024*1024)} MB remaining")

# Completion callback
def on_complete(stream, file_handle):
    print("Downloaded!")

youtube_clip = YouTube(video_url,
                       on_progress_callback=on_progress,
                       on_complete_callback=on_complete)

youtube_clip.streams.get_highest_resolution().download("videos")